In [13]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime

# Example folder path
folder_path = '../NASAResources/space_apps_2024_seismic_detection/data/mars/training/data/'
# Load the catalog (starting points of quakes)
catalog_path = '../NASAResources/space_apps_2024_seismic_detection/data/mars/training/catalogs/Mars_InSight_training_catalog_final.csv'
catalog = pd.read_csv(catalog_path)


In [14]:
# Function to reformat the absolute time by setting the fractional seconds to .00000
def reformat_abs_time(time_str):
    dt = datetime.strptime(time_str, '%Y-%m-%dT%H:%M:%S.%f')
    reformatted_time = dt.strftime('%Y-%m-%dT%H:%M:00') + '.000000'
    return reformatted_time

def reformat_data_file(data):
    # Apply the reformatting function to the 'time_abs' column
    # FOR LUNAR 
    #data['time_abs'] = data['time_abs(%Y-%m-%dT%H:%M:%S.%f)'].apply(reformat_abs_time)
    # FOR MARS
    data['time_abs'] = data['time(%Y-%m-%dT%H:%M:%S.%f)'].apply(reformat_abs_time)
    # Processing data folder
    #FOR LUNAR 
    #data['time_rel'] = data['time_rel(sec)'].astype(int)
    #FOR MARS
    data['time_rel'] = data['rel_time(sec)'].astype(int)
    # FOR LUNAR 
    #data['velocity'] = data['velocity(m/s)']
    # FOR MARS
    data['velocity'] = data['velocity(c/s)']
    return data

#features = data[['time_rel', 'velocity', 'time_abs']]

catalog['time_rel'] = catalog['time_rel(sec)'].astype(int)
catalog['time_abs'] = catalog['time_abs(%Y-%m-%dT%H:%M:%S.%f)']


In [15]:
#print(data['time_rel'].iloc[50])

In [16]:
def label_data(data, catalog):
    # Create a boolean mask for matching time_rel
    mask_rel = data['time_rel'].isin(catalog['time_rel'])
    filtered_data = data[mask_rel]
    filtered_catalog = catalog[catalog['time_rel'].isin(filtered_data['time_rel'])]
    # Create a boolean mask for matching time_abs in the filtered data
    mask_abs = filtered_data.apply(lambda row: row['time_abs'] in filtered_catalog[filtered_catalog['time_rel'] == row['time_rel']]['time_abs'].values, axis=1)
    # Initialize labels array with zeros
    labels = np.zeros(len(data), dtype=int)
    # Set labels to 1 where both masks are True
    labels[mask_rel[mask_rel].index[mask_abs]] = 1
    return labels

# Iterate over all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        # Load CSV data
        data = pd.read_csv(file_path)
        # Reformat the data file for finding the matching time_abs & time_rel
        reformat_data_file(data)
        # Apply labeling
        labels = label_data(data, catalog)
        # Add labels as a new column to the DataFrame
        data['labels'] = labels
        # Save the updated DataFrame back to the CSV file
        data.to_csv(file_path, index=False)

print("Labeling completed for all CSV files in the folder.")



PermissionError: [Errno 13] Permission denied: '../NASAResources/space_apps_2024_seismic_detection/data/mars/training/data/XB.ELYSE.02.BHV.2022-01-02HR04_evid0006.csv'

In [16]:
#quake_start_count = np.sum(labels)
#print(quake_start_count)